# AFAP Ratio Engine Validation

This notebook validates:
1. Deterministic ratio calculations from `ratio_engine_core`
2. Threshold-based interpretation from `ratio_engine_eval`
3. Schema integrity and orchestration readiness

This notebook is part of AFAP Phase 3 (Locked Engines).


In [ ]:
import pandas as pd
import numpy as np
import sys
import os

from engines.ratio_engine_core import ratio_engine
from engines.ratio_engine_eval import evaluate_ratios
from engines.schema_validator import validate_engine_output


ModuleNotFoundError: No module named 'engines'